# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [383]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [384]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [460]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts=Counter(text)  #put all text in a Counter with their frequencies
    vocab = sorted(counts, key=counts.get, reverse=True)  #sort by highest frequency to lowest
    #dictionary vocab_to_int, word is key, index int is value
    #vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    #int to vocab is a dictionary the other way around, ii is key, word is index
    #int_to_vocab={ii:word for ii, word in enumerate(vocab,1)}
    
    #why does the above code not work?
    int_to_vocab = {index: word for index, word in enumerate(vocab)}
    vocab_to_int = {word: index for index, word in int_to_vocab.items()}
    
    print(vocab_to_int)
    print(int_to_vocab)
    return vocab_to_int, int_to_vocab
    
    #return None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

{'where': 21, 'one': 13, 'homer_simpson': 23, 'another': 24, 'yeah': 7, "moe's": 25, 'listen': 57, 'name': 9, 'of': 28, 'to': 6, 'these': 29, 'my': 11, 'hey': 12, 'gonna': 8, 'the': 14, 'carve': 30, 'normal': 31, 'self': 32, 'drink': 5, 'hello': 69, 'last': 67, 'enhance': 33, 'me': 66, 'matter': 34, 'meet': 35, 'little': 39, 'with': 59, 'back': 60, 'catch': 70, 'got': 38, 'not': 16, 'forget': 20, 'anybody': 40, 'puke': 48, 'whats': 68, 'moe': 27, 'has': 44, 'seen': 45, 'there': 46, 'your': 1, 'lately': 47, 'give': 41, 'effervescent': 36, 'pick': 50, 'only': 37, 'mike': 2, 'eh': 52, 'on': 17, 'skills': 54, 'days': 55, 'hold': 56, 'moe_szyslak': 0, 'should': 10, 'an': 26, "i'll": 22, 'problems': 15, 'ice': 42, 'i': 61, 'barney_gumble': 51, 'social': 63, 'is': 49, 'elite': 64, 'and': 65, "i'm": 18, 'bart_simpson': 58, 'homer': 19, "you're": 43, 'tavern': 53, 'rotch': 3, 'check': 62, 'you': 4}
{0: 'moe_szyslak', 1: 'your', 2: 'mike', 3: 'rotch', 4: 'you', 5: 'drink', 6: 'to', 7: 'yeah', 8:

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [461]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    #key is punctuation, value is the token
    
    punc_dict = {}
    punc_dict['.'] = "||period||"
    punc_dict[','] = "||comma||"
    punc_dict['"'] = "||quotation_mark||"
    punc_dict[';'] = "||semicolon||"
    punc_dict['!'] = "||exclamation_mark||"
    punc_dict['?'] = "||question_mark||"
    punc_dict['('] = "||left_parantheses||"
    punc_dict[')'] = "||right_parantheses||"
    punc_dict['--'] = "||dash||"
    punc_dict['\n'] = "||return||"
    print(punc_dict.items())

    return punc_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

dict_items([('(', '||left_parantheses||'), ('\n', '||return||'), (';', '||semicolon||'), ('!', '||exclamation_mark||'), (',', '||comma||'), ('"', '||quotation_mark||'), (')', '||right_parantheses||'), ('.', '||period||'), ('?', '||question_mark||'), ('--', '||dash||')])
Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [462]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

dict_items([('(', '||left_parantheses||'), ('\n', '||return||'), (';', '||semicolon||'), ('!', '||exclamation_mark||'), (',', '||comma||'), ('"', '||quotation_mark||'), (')', '||right_parantheses||'), ('.', '||period||'), ('?', '||question_mark||'), ('--', '||dash||')])
{'photo': 1900, 'dana_scully:': 2896, 'cocktail': 1440, 'washouts': 5237, 'girl-bart': 2898, "america's": 2899, 'musical': 2900, 'ad': 961, 'disaster': 2901, 'wipes': 2902, 'thanking': 2903, 'refreshment': 2904, 'greetings': 2519, 'poker': 2309, 'pint': 1901, 'urban': 2907, 'playing': 604, 'furry': 4717, 'gangrene': 2909, "england's": 5154, 'bucks': 379, 'funniest': 2911, 'test-': 2912, 'he': 75, 'enjoy': 1170, 'pays': 1441, 'agents': 2913, 'luck': 962, '1895': 1902, 'course': 513, 'winner': 2914, 'walked': 2915, 'right-handed': 2916, 'ducked': 5524, 'wanted': 408, 'kid': 289, 'lucky': 664, 'ford': 2919, 'puzzle': 2920, 'outrageous': 2921, 'mistresses': 2922, 'tape': 514, 'liability': 2923, 'compared': 1927, 'someplace'

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [463]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [464]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/Users/qamrakhan/anaconda3/envs/dlnd-tf-lab/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [465]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape= [None, None], name='input')
    targets = tf.placeholder(tf.int32, shape= [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return (inputs, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [466]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #basic LSTM cell
    lstmCell=tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    #stack up multiple LSTM layers.
    lstm_layers=1  #can change this later
    #[lstm]*lstm_layers creates a list of cells that is lstm layers long
    #stack up multiple LSTM layers (for now, lstm_layers=1)
    multiCell=tf.contrib.rnn.MultiRNNCell([lstmCell]*lstm_layers)  #no point as lstm_layers=1 for now
    #MultiCell=tf.contrib.rnn.MultiRNNCell([LSTMCell])
    #get an initial state of all zeros.
    initial_state=multiCell.zero_state(batch_size, tf.float32)
    #tf.identity: useful when you need to transport tensors. can be used as a dummy node to update a reference to a tensor
    initial_state=tf.identity(initial_state, name='initial_state')
    
    #InitialState = tf.identity(Cell.zero_state(batch_size, tf.float32), name="initial_state")
    return (multiCell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [467]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    #use an embedding layer as a lookup table
    #embed_dim number of features dimensions, which will group context of words by
    embedding=tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    #tf.nn.embedding_lookup: returns a tensor with the embedded vectors, size[batch_size,embed_dim]
    embed=tf.nn.embedding_lookup(embedding,input_data)
    #print(input_data, vocab_size, embed_dim)
    #print(embedding)
    #print(embed_dim,tf.shape(embed))
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [468]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    #run data through RNN nodes. 
    #in this case inputs is actually from the embed layer
    #initial_state passed between RNNs for each time step.
    #tf.nn.dynamic_rnn: returns outputs for each timestep, and the final state of the hidden layer
    #tf.nn.dynamic_rnn:no initial_state provided, so dtype must be provided
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state=tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [469]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    #apply embedding to input_data using get_embed function
    #get_embed(input_data, vocab_size, embed_dim):
    #embedded vectors are inputs into LSTM layer
    rnn_inputs=get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state =build_rnn(cell,rnn_inputs)
    #fully connected layer and predictions
    #in this case care about all outputs not only final output (like sentiment RNN)
    
    #Logits=tf.contrib.layers.fully_connected(Outputs,vocab_size,activation_fn=tf.sigmoid)
    #linear activation, not sigmoid!
    logits=tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None)
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [470]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    # find total number of batches to make. batch_size*sequence length in this case as we care about sequence
    #print(int_text[:20], len(int_text))
    n_batches = len(int_text)//(batch_size*seq_length)
    #print(n_batches, batch_size, seq_length)
    #n_batches=int(n_batches)   #int rounds down, so ignore last batch if not complete
    
    batches=np.zeros((n_batches,2,batch_size,seq_length))
    
    #last target of last batch should be 1st input of first batch
    #means int_text[n_batches*batch_size*seq_length+1]=int_text[0]
    #print(n_batches*batch_size*seq_length,int_text[0])  #no +1 as starts from [0]
    #int_text[n_batches*batch_size*seq_length]=int_text[0]
    
    for i in range(n_batches*batch_size*seq_length, len(int_text)):
        int_text[i] = int_text[i-n_batches*batch_size*seq_length]
        
    for i in range(n_batches):      #batch_size=128
        for j in range(batch_size):   #n_batches=7
            
            #below is fine if fill up batches in order: e.g. split all of batch 1, batch 2, batch3
            start_i=j*n_batches*seq_length+i*seq_length
            #i*seq_length + j*n_batches*seq_length
            end_i=start_i+seq_length
            #input batch
            batches[i][0][j]=int_text[start_i:end_i]
            #print(i,j,batches[i][0][j])
            #target batch
            batches[i][1][j]=int_text[start_i+1:end_i+1]



    return np.array(batches)


    #for ii in range(0,n_batches,1):
    #    print(ii)
    #    yield np.array([n_batches,x[ii:ii+batch_size*seq_length],2,seq_length],dtype=np.int32)
    
    #for ii in range(0,len(int_text),batch_size*seq_length):
    #    print(ii)
    #    yield np.array([n_batches,x[ii:ii+batch_size*seq_length],2,seq_length])
    
    #The batches should be a Numpy array with the shape (number of batches, 2, batch size, sequence length)
    
    
    
    
    #return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [491]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 25
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 100

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [492]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [493]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/10   train_loss = 8.821
Epoch  10 Batch    0/10   train_loss = 2.857
Epoch  20 Batch    0/10   train_loss = 1.619
Epoch  30 Batch    0/10   train_loss = 1.010
Epoch  40 Batch    0/10   train_loss = 0.772
Epoch  50 Batch    0/10   train_loss = 0.457
Epoch  60 Batch    0/10   train_loss = 0.369
Epoch  70 Batch    0/10   train_loss = 0.224
Epoch  80 Batch    0/10   train_loss = 0.194
Epoch  90 Batch    0/10   train_loss = 0.137
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [499]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [500]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [501]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    init_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0')
    return input_tensor, init_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [502]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #get the index of the max probability
    max_prob=np.argmax(probabilities)
    print(probabilities)
    print(max_prob,int_to_vocab[max_prob])
    #int_to_vocab={word_int_id:words}
    return int_to_vocab[max_prob]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

[ 0.1   0.8   0.05  0.05]
1 is
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [503]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

[  1.10708608e-03   4.40181157e-06   3.49280919e-04 ...,   9.06444786e-09
   4.03642360e-07   2.68730851e-08]
3 ||left_parantheses||
[  1.55084265e-07   3.29474847e-09   7.66087487e-06 ...,   5.22458576e-10
   8.69615047e-09   2.73212009e-09]
287 sighs
[  3.38123057e-13   2.18670798e-13   8.56687592e-08 ...,   1.37479006e-17
   2.09279098e-15   3.43734076e-17]
4 ||right_parantheses||
[  3.21248024e-07   6.57182280e-03   6.53573716e-06 ...,   1.82722237e-09
   4.12041790e-10   1.37853002e-11]
6 i
[  9.72172165e-06   2.53094044e-08   4.35793336e-05 ...,   6.28036234e-11
   6.13112805e-13   1.54290747e-11]
169 didn't
[  5.68504678e-03   6.58563259e-10   5.65229584e-07 ...,   1.04176487e-12
   9.78618644e-11   2.16975504e-08]
101 want
[  8.52160520e-05   3.23105875e-10   9.19966360e-12 ...,   1.25999157e-12
   8.97958010e-12   8.88636942e-11]
13 to
[  4.51508095e-05   1.73354520e-09   6.03584427e-09 ...,   1.37914419e-11
   4.49950217e-13   4.20684820e-09]
47 have
[  2.47636810e-03   4.309

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.